In [25]:
# coding:u8
import pandas as pd
import numpy as np
import copy,gc
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import f1_score,classification_report
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
import seaborn as sns



In [26]:
data = pd.read_pickle('features/1_stat_feature.pkl')

In [27]:
# data = data.fillna(-999)

In [28]:
data.head()

,text_id,kb_id,train_mention,label,num_attrs,num_abstract_words,num_alias,m_id,num_candidates,mention_equal_subject,...,rank_num_abstract_words,rank_num_alias,rank_label_mean,rank_label_count,rank_m_label_mean,rank_m_label_count,rank_num_candidates,rank_mention_equal_subject,rank_shuminghao,rank_entity_common
0,1,311223,南京南站,1,17,447,3,0,2,True,...,1.0,1.0,NaN,NaN,NaN,NaN,1.5,1.5,1.5,1.5
1,1,130287,南京南站,0,9,338,2,0,2,True,...,2.0,2.0,NaN,NaN,NaN,NaN,1.5,1.5,1.5,1.5
2,1,213561,高铁,0,1,0,2,1,2,False,...,2.0,1.0,2.0,1.5,2.0,1.5,1.5,2.0,1.5,2.0
3,1,341096,高铁,1,10,239,1,1,2,True,...,1.0,2.0,1.0,1.5,1.0,1.5,1.5,1.0,1.5,1.0
4,1,311223,南京南站,1,17,447,3,2,2,True,...,1.0,1.0,NaN,NaN,NaN,NaN,1.5,1.5,1.5,1.5


In [29]:

predictor = ['num_attrs',
       'num_abstract_words', 'num_alias', 'num_candidates',
       'mention_equal_subject', 'shuminghao', 'len_mention', 'mention_start',
       'academicdiscipline', 'animal', 'astronomicalobject', 'athlete',
       'awardeventseries', 'brand', 'building', 'collegeoruniversity',
       'communicationmedium', 'country', 'creativework', 'culturalheritage',
       'currency', 'curriculum', 'educationmajor', 'entertainmentperson',
       'event', 'familyname', 'fictionalhuman', 'fictionalthing', 'food',
       'formula', 'game', 'historicalperiod', 'historicalperson', 'human',
       'internationalorganization', 'language', 'material', 'medicalcondition',
       'medicaldepartmenttype', 'movie', 'nation', 'organism', 'organization',
       'person', 'place', 'plant', 'product', 'scientificorganization',
       'symbol', 'theorem', 'thing', 'tool', 'tvplay', 'tvshow', 'vocabulary',
       'zodiacsign', 'label_mean', 'label_count', 'm_label_mean',
       'm_label_count', 'rank_num_attrs', 'rank_num_abstract_words',
       'rank_num_alias', 'rank_label_mean', 'rank_label_count',
       'rank_m_label_mean', 'rank_m_label_count', 'rank_num_candidates',
       'rank_mention_equal_subject', 'rank_shuminghao', 'rank_entity_common','entity_common']

In [30]:
exclude = []
# exclude = []
temp = []
for x in predictor:
    if x not in exclude:temp.append(x)
predictor2 = temp

In [31]:
param = {
    #'num_leaves':100,
    'objective':'binary',
    #'metric':'acc',
    'metric_freq':20,
    'learning_rate': 0.05,
    'num_threads':8,
    #'min_sum_hessian_in_leaf':10,
    'boosting_type':'gbdt',
    'subsample':0.9,
    'colsample_bytree':0.8,
    'n_estimators':20000,
    'min_child_weight':1,
    'subsample_freq':2,
    'num_leaves':128,
    'n_jobs':-1,
}


In [ ]:
kfold = KFold(n_splits=9,shuffle=False,random_state=2019)
for train_index,test_index in kfold.split(data):
    train_data,val_data = data.loc[train_index,:],data.loc[test_index,:]
    train_data = train_data.fillna(-999)
    train = lgb.Dataset(train_data[predictor2],label=train_data['label'])
    validation_data = lgb.Dataset(val_data[predictor2],label=val_data['label'])
    
    val_data.reset_index(inplace=True,drop=True)
    val_data.reset_index(inplace=True)
    index_df = val_data.groupby('m_id')['index'].apply(list)
    
    label_package = val_data.groupby('m_id',as_index=False)['label'].agg({'label_id_max':'idxmax'})['label_id_max']
    
    def lgb_f1_score(y_hat, data):
        y_true = data.get_label()
        hit = 0
        for index,pack in enumerate(index_df):
            score = []
            for i in pack:
                score.append(y_hat[i])
            if pack[np.argmax(score)]==label_package[index]:
                hit+=1
        best_score = hit/len(index_df)
        return 'acc', best_score, True
    bst = lgb.train(param, train, valid_sets=[validation_data],early_stopping_rounds=500,
                    verbose_eval=20,feval=lgb_f1_score)
#     break
# 0.8672
# 0.872
# [171]	valid_0's binary_logloss: 0.0988336	valid_0's acc: 0.873219
# valid_0's binary_logloss: 0.0964686	valid_0's acc: 0.87713
# [224]	valid_0's binary_logloss: 0.096617	valid_0's acc: 0.877515
# [308]	valid_0's binary_logloss: 0.09149	valid_0's acc: 0.890507 entity common
# drop ctr 0.877
# [2094]	valid_0's binary_logloss: 0.0791409	valid_0's acc: 0.896452
# [1674]	valid_0's binary_logloss: 0.0784442	valid_0's acc: 0.896347
#０．９１５
# [1344]	valid_0's binary_logloss: 0.0767459	valid_0's acc: 0.897395
# [405]	valid_0's binary_logloss: 0.0681472	valid_0's acc: 0.913129
# [1600]	valid_0's binary_logloss: 0.0672695	valid_0's acc: 0.91415


In [17]:
############################## for submit  #####################################

train = lgb.Dataset(data[predictor2],label=data['label'])
param = {
    #'num_leaves':100,
    'objective':'binary',
    #'metric':'acc',
    'metric_freq':20,
    'learning_rate': 0.05,
    'num_threads':8,
    #'min_sum_hessian_in_leaf':10,
    'boosting_type':'gbdt',
    'subsample':0.9,
    'colsample_bytree':0.8,
    'n_estimators':1700,
    'min_child_weight':1,
    'subsample_freq':2,
    'num_leaves':128,
    'n_jobs':-1,
}

bst = lgb.train(param, train)
from sklearn.externals import joblib
joblib.dump(bst, 'model_link/lgb.pkl')

# save model
# load model
#gbm_pickle = joblib.load('lgb.pkl')
# break

KeyboardInterrupt: 

In [ ]:
# val_data

In [ ]:
fold_importance_df = pd.DataFrame()
fold_importance_df["feature"] = predictor2
fold_importance_df["importance"] = bst.feature_importance()
cols = (fold_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = fold_importance_df.loc[fold_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

In [ ]:
fold_importance_df = pd.DataFrame()
fold_importance_df["feature"] = predictor2
fold_importance_df["importance"] = bst.feature_importance()
cols = (fold_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = fold_importance_df.loc[fold_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')